# In-class exercise 7: Deep Learning (Part 1A)
In this notebook we will see how to write efficient and numerically stable code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import minmax_scale

In [7]:
X, y = load_breast_cancer(return_X_y=True)
# Scale each feature to [-1, 1] range
X = minmax_scale(X, feature_range=(-1, 1))
X.shape, y.shape



((569, 30), (569,))

# 1. Vectorization

## 1.1. Logistic regression (two classes)

**Setting:** Logistic regression (two classes)

**Task:** Generate predictions for the entire dataset

In [3]:
n_features = X.shape[1]   #D from N*D
w = np.random.normal(size=[n_features], scale=0.1)  # weight vector
b = np.random.normal(size=[1])  # bias





In [8]:
def sigmoid(t):
    """Apply sigmoid to the input array."""    
    return 1 / (1 + np.exp(-t))



### Bad - for loops

In [9]:
def predict_for_loop(X, w, b):
    """Generate predictions with a logistic regression model using a for-loop.
    
    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)
        
    Returns:
        y: probabilies of the positive class, shape (N)
    """
    n_samples = X.shape[0]
    y = np.zeros([n_samples])
    for i in range(n_samples):
        score = np.dot(X[i], w) + b
        y[i] = sigmoid(score)
    return y

In [11]:
X.shape, w.shape, b.shape

((569, 30), (30,), (1,))

In [12]:
(X@w).shape

(569,)

### Good - vectorization

In [13]:
def predict_vectorized(X, w, b):
    """Generate predictions with a logistic regression model using vectorized operations.
    
    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)
        
    Returns:
        y: probabilies of the positive class, shape (N)
    """
    scores = X @ w + b   #shape(n_samples)
    y = sigmoid(scores)
    return y





### Compare the runtime of two variants

In [14]:
%%timeit
predict_for_loop(X, w, b)



4.92 ms ± 99 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
predict_vectorized(X, w, b)




22.9 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 1.2. K-nearest neighbors
A more complicated task: compute the matrix of pairwise distances.

Given a data matrix `X` of size `[N, D]`, compute the matrix `dist` of pairwise distances of size `[N, N]`, where `dist[i, j] = l2_distance(X[i], X[j])`.

### Bad - for loops

In [16]:
def l2_distance(x, y):
    """Compute Euclidean distance between two vectors."""
    return np.sqrt(np.sum((x - y) ** 2))




In [19]:
def distances_for_loop(X):
    """Compute pairwise distances between all instances (for loop version).
    
    Args:
        X: data matrix, shape (N, D)
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    n_samples = X.shape[0]
    distances = np.zeros([n_samples, n_samples])
    for i in range(n_samples):
        for j in range(n_samples):
            distances[i, j] = l2_distance(X[i], X[j])
    return distances

#O(|N|^2)



In [22]:
dist1 = distances_for_loop(X)
dist1.shape



(569, 569)

### Good - vectorization

How can we compute all the distances in a vectorized way?

Start with a simpler example.

In [24]:
x = np.arange(5, dtype=np.float64)
print(x)



[0. 1. 2. 3. 4.]


In [27]:
x.reshape([-1,1])   

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.]])

In [28]:
x.reshape([1, -1])

array([[0., 1., 2., 3., 4.]])

In [29]:
x.reshape([-1,1]) - x.reshape([1, -1]) #x.reshape([-1, 1]) boardcast to [5,5]


array([[ 0., -1., -2., -3., -4.],
       [ 1.,  0., -1., -2., -3.],
       [ 2.,  1.,  0., -1., -2.],
       [ 3.,  2.,  1.,  0., -1.],
       [ 4.,  3.,  2.,  1.,  0.]])

In [32]:
#boardcasting equal to
x.reshape([-1,1]).repeat(5, axis=1)




array([[0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4.]])

In [34]:
x.reshape([1,-1]).repeat(5, axis=0)

array([[0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.]])

In [59]:
#turn x-shape(5,) to shape(5,1)
x2 = np.arange(5)
x2.shape
x2[:, None].shape #-> shape(5,1) 
x2[:, np.newaxis].shape#all entry at the 0-axis and for the first axis, add an additional dimension



(5, 1)

Increase the dimension of an array using `np.newaxis`

In [35]:
x[:, np.newaxis]

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.]])

In [36]:
x[np.newaxis, :] - x[:, np.newaxis]

array([[ 0.,  1.,  2.,  3.,  4.],
       [-1.,  0.,  1.,  2.,  3.],
       [-2., -1.,  0.,  1.,  2.],
       [-3., -2., -1.,  0.,  1.],
       [-4., -3., -2., -1.,  0.]])

In [38]:
def distances_vectorized(X):
    """Compute pairwise distances between all instances (vectorized version).
    
    Args:
        X: data matrix, shape (N, D)
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    return np.sqrt(((X[:, None] - X[None, :])**2).sum(-1))


def distances_vectorized2(X):
    """Compute pairwise distances between all instances (vectorized version).
    
    Args:
        X: data matrix, shape (N, D)
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    return np.sqrt(((X - X)**2).sum(-1))
#sum(-1) take the rightest dimension



In [77]:
X[:, None,:].shape



(569, 1, 30)

In [78]:
X[:, None].shape

(569, 1, 30)

In [67]:
X[None, :, :].shape

(1, 569, 30)

In [71]:
(X[:, None,:]-X[None,:,:]).shape #N*N*D   #[i,j,k] = X[i,k]-X[j,k]

(569, 569, 30)

In [79]:
print((X[:, None] - X[None, :]).shape)
y1=((X[:, None] - X[None, :])**2).sum(-1)
y2=((X[:, None] - X[None, :])**2).sum(2)
np.all(y1==y2)



(569, 569, 30)


True

In [94]:
dist2 = distances_vectorized(X)
dist22 = distances_vectorized2(X)
np.all(dist2==dist22)
dist2.shape
    


(569, 569)

Make sure that both variants produce the same results

In [95]:
# Direct comparison fails because of tiny numerical differences
np.all(dist1 == dist2)



False

In [96]:
# Two results are very close
np.linalg.norm(dist1 - dist2, ord='fro')




2.382806063356817e-14

In [97]:
# Use np.allclose to compare
np.allclose(dist1, dist2)




True

### Best - library function

In [98]:
#!!!!!!!!!!!
from scipy.spatial.distance import cdist, pdist, squareform
# Compute distance between each pair of the two collections of inputs
# 两个矩阵行间的所有向量对的距离
dist3 = cdist(X, X)
# pdist:Pairwise distances between observations in n-dimensional space.
# 所有向量间的距离
dist4 = squareform(pdist(X))  #every pari of points





In [99]:
np.all(dist3==dist4)

True

In [100]:
np.all(dist3==dist2)

False

In [101]:
np.linalg.norm(dist2-dist3, ord='fro') #Frobenius norm

1.1201143480289869e-13

### Compare the runtime

In [83]:
%%timeit
dist1 = distances_for_loop(X)



3.19 s ± 55.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%%timeit
dist2 = distances_vectorized(X)



44.4 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [104]:
%%timeit
dist3 = cdist(X, X)



7.47 ms ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [105]:
%%timeit
dist4 = squareform(pdist(X))




4.43 ms ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Lessons:
1. For-loops are extremely slow! Avoid them whenever possible.
2. A better alternative - use matrix operations & broadcasting
3. An even better alternative - use library functions (if they are available).
4. Implementations with for-loops can be useful for debugging vectorized code.

# 2. Numerical stability
Typically, GPUs use single precision (32bit) floating point numbers (in some cases even half precision / 16bit). This significantly speeds ups the computations, but also makes numerical issues a lot more likely. 
Because of this we always have to be extremely careful to implement our code in a numerically stable way.

Most commonly, numerical issues occur when dealing with `log` and `exp` functions (e.g. when computing cross-entropy of a categorical distribution) and `sqrt` for values close to zero (e.g. when computing standard deviations or normalizing the $L_2$ norm).

## 2.1. Avoiding numerical overflow (exploding `exp`)

Softmax function $f : \mathbb{R}^D \to \Delta^{D - 1}$ converts a vector $\mathbf{x} \in \mathbb{R}^D$ into a vector of probabilities.

$$f(\mathbf{x})_j = \frac{\exp(x_j)}{\sum_{d=1}^{D} \exp(x_d)}$$

Apply the softmax function to the following vector.

In [106]:
x = np.linspace(0., 4., 5).astype(np.float32)#numpy.linspace(start, stop, num=50)
x




array([0., 1., 2., 3., 4.], dtype=float32)

In [109]:
# Our code here
denominator=np.exp(x).sum()
np.exp(x) / denominator

    

array([0.01165623, 0.03168492, 0.08612855, 0.23412167, 0.6364086 ],
      dtype=float32)

Now apply it to the following vector

In [110]:
x = np.linspace(50., 90., 5).astype(np.float32)
x



array([50., 60., 70., 80., 90.], dtype=float32)

In [113]:
np.exp(x)

/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


array([5.1847055e+21, 1.1420074e+26, 2.5154387e+30, 5.5406225e+34,
                 inf], dtype=float32)

In [111]:
# Our code here
denominator = np.exp(x).sum()
np.exp(x) / denominator



/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([ 0.,  0.,  0.,  0., nan], dtype=float32)

How to avoid the exposion?

In [112]:
x_shifted = x - np.max(x)
denominator = np.exp(x_shifted).sum()
np.exp(x_shifted) / denominator





array([4.248161e-18, 9.357198e-14, 2.061060e-09, 4.539787e-05,
       9.999546e-01], dtype=float32)

## 2.2. Working in the log-space / simplifying the expressions

Binary cross entropy (BCE) loss for a logistic regression model (corresponds to negative log-likelihood of a Bernoulli model)

$$\log p(\mathbf{y} \mid \mathbf{X}, \mathbf{w}, b) = -\sum_{i=1}^{N} y_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) + (1 - y_i) \log (1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b))$$


Implement the BCE computation.

In [114]:
# TODO
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

def binary_cross_entropy_unstable(scores, labels):
    return -labels * np.log(sigmoid(scores)) - (1 - labels) * np.log(1 - sigmoid(scores))




In [115]:
x = np.array([[20., 20.]])  #shape (1,2)
w = np.array([[1., 1.]])
y = np.array([1.])

scores = x @ w.T
binary_cross_entropy_unstable(scores, y)





/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


array([[nan]])

In [116]:
np.log(1-sigmoid(scores))

/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


array([[-inf]])

In [117]:
(1-y)*np.log(1-sigmoid(scores))

/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/Users/xiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([[nan]])

Try to simplify the BCE loss as much as possible

In [32]:
# TODO
def binary_cross_entropy_stable(scores, labels):
    return np.log(1 + np.exp(scores)) - labels * scores



binary_cross_entropy_stable(scores, y)

array([[0.]])

## 2.3. Loss of numerical precision

Implement the log sigmoid function 

$$f(x) = \log \sigma(x) = \log \left(\frac{1}{1 + \exp(-x)}\right) = log 1 -log(1+exp(-x))=-log(1+exp(-x))$$




In [119]:
# Your code here
def log_sigmoid_unstable(x):
    return np.log(1 / (1 + np.exp(-x)))



`float32` has much lower "resolution" than `float64`

In [120]:
x = np.linspace(0, 30, 11).astype(np.float32) #can only present a small fraction of the numbers
log_sigmoid_unstable(x)
#less accurate prediction




array([-6.9314718e-01, -4.8587345e-02, -2.4756414e-03, -1.2338923e-04,
       -6.1989022e-06, -3.5762793e-07,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32)

In [121]:
x = np.linspace(0, 30, 11).astype(np.float64)
log_sigmoid_unstable(x)




array([-6.93147181e-01, -4.85873516e-02, -2.47568514e-03, -1.23402190e-04,
       -6.14419348e-06, -3.05902274e-07, -1.52299796e-08, -7.58256125e-10,
       -3.77513576e-11, -1.87960758e-12, -9.34807787e-14])

Implement the log-sigmoid function in a numerically stable way

In [122]:
def log_sigmoid_stable(x):
    return -np.log1p(np.exp(-x)) #log1p: log(1 + x)

#better then -np.log(1+np.exp(-x))




In [123]:
x = np.linspace(0, 30, 11).astype(np.float32)
log_sigmoid_stable(x)
#more accurate but using single precision float



array([-6.9314718e-01, -4.8587348e-02, -2.4756852e-03, -1.2340219e-04,
       -6.1441933e-06, -3.0590226e-07, -1.5229979e-08, -7.5825607e-10,
       -3.7751344e-11, -1.8795289e-12, -9.3576229e-14], dtype=float32)

In [124]:
np.log1p? #log(1 + x)

Relevant functions: `np.log1p`, `np.expm1`, `scipy.special.logsumexp`, `scipy.special.softmax` -- these are also implemented in all major deep learning frameworks.

`np.exmp1`: Calculate exp(x) - 1 for all elements in the array 

`scipy.special.logsumexp`: Compute the log of the sum of exponentials of input elements.

`scipy.special.softmax` : softmax(x) = np.exp(x)/sum(np.exp(x))

different types of numerical instability: underflow and overflow, if we try to exponentiate by the very large or samll numbers.

always try to simply the loss function 

softmax function: substract the maximum value to avoid overflow



## Lessons:
1. Be especially careful when working with `log` and `exp` functions in **single precision** floating point arithmetics
2. Work in the log-space when possible
3. Use numerically stable library functions when available